# Tarea Automatas y Compiladores - ICI3245/1
Integrantes:
- Alvaro Del Pino
- Javier Morales
- Matias Ruiz
- Sebastián Rojas

## Instrucciones:
Diseñar un nuevo lenguaje de programación e implementar su analizador léxico, sintáctico, semántico y generación de código. El lenguaje debe soportar como mínimo los siguientes elementos:
  - Variables.
  - 3 tipos de datos.
  - impresión.
  - Una estructura condicional.
  - 2 estructuras repetitivas.
  - 2 operadores lógicos.
  - Operadores matemáticos básicos (+,-,/,*).
  - 4 funciones matemáticas (sqrt, cos, sin, etc.).
  - 3 reglas semánticas a elección. Ejemplos:
      - Detección de Variables Declaradas y No Usadas
      - Validar Duplicidad de Nombres de Métodos en una Clase
      - Comprobar que Métodos de Tipo void no Retornen Valor
      - Validar Nombres de Variables (Convención de Estilo)
      - Detección de Métodos sin Instrucciones de Retorno

La tarea debe contener:
  - Un archivo .txt con la gramática del lenguaje (en notación E-BNF).
  - Un archivo .txt incluyendo un ejemplo ilustrativo del lenguaje.
  - Un archivo .zip con el proyecto(Python(colab) + fuentes .g4)

Consideraciones:
  - Se valorará originalidad del lenguaje (Este NO puede ser una copia del lenguaje visto
  en clases).
  - Subir la tarea al Aula, Viernes 20 de junio hasta las 12 del medio día.
  - Máximo 5 personas por grupo.


# Introducción

Un compilador es un programa que traduce código fuente escrito en un lenguaje de alto nivel (como Java, C, Python) a un formato ejecutable o a un código intermedio. Este proceso implica varias fases:

Análisis Léxico (Lexer): Convierte el código fuente en tokens (las palabras básicas del lenguaje).
Análisis Sintáctico (Parser): Verifica la estructura del código a partir de las reglas gramaticales.
Análisis Semántico: Valida el significado del código, como comprobar que las variables estén declaradas antes de usarse.
Generación de Código: Crea un código de salida, que puede ser código de máquina, bytecode o un pseudocódigo intermedio.
Fuente (Java) │ ▼ Lexer (Tokens) → Parser (AST) → Visitor/Listener (Análisis Semántico) → Generación de Código

**OBJETIVO:**
Nuestro objetivo será crear un lenguaje de programación diseñado con fines educativos, orientado a facilitar el aprendizaje de conceptos fundamentales de programación estructurada, mientras permite trabajar con expresiones y funciones matemáticas. Su sintaxis es simple y clara, pensada para principiantes o entornos académicos donde se quiere enseñar lógica de programación, álgebra computacional básica y estructuras de control.

In [ ]:
# Instalamos ANTLR y descargamos el jar para generar el parser
!pip install antlr4-python3-runtime==4.9.2
!pip install graphviz
!apt-get install graphviz
!wget https://www.antlr.org/download/antlr-4.9.2-complete.jar -O antlr-4.9.2-complete.jar


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
graphviz is already the newest version (2.42.2-6ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
--2025-06-20 00:32:51--  https://www.antlr.org/download/antlr-4.9.2-complete.jar
Resolving www.antlr.org (www.antlr.org)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to www.antlr.org (www.antlr.org)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2100564 (2.0M) [application/java-archive]
Saving to: ‘antlr-4.9.2-complete.jar’

antlr-4.9.2-complet 100%[===================>]   2.00M  --.-KB/s    in 0.07s   

2025-06-20 00:32:52 (28.4 MB/s) - ‘antlr-4.9.2-complete.jar’ saved [2100564/2100564]



In [ ]:
%%writefile LenguajeEdu.g4
grammar LenguajeEdu;

// ======== Parser ========

programa: instruccion* EOF ;

instruccion
    : declaracion
    | impresion
    | condicional
    | bucle_mientras
    | bucle_para
    | funcion_llamada PUNTOYCOMA
    | retorno
    | asignacion
    | PUNTOYCOMA
    ;

declaracion
    : tipo_dato IDENTIFICADOR (ASIGNACION expr)? PUNTOYCOMA
    ;

asignacion
    : IDENTIFICADOR ASIGNACION expr PUNTOYCOMA
    ;

impresion
    : IMPRIMIR PAR_IZQ (expr | CADENA) PAR_DER PUNTOYCOMA
    ;

condicional
    : SI PAR_IZQ expr PAR_DER bloque (SINO bloque)?
    ;

bucle_mientras
    : MIENTRAS PAR_IZQ expr PAR_DER bloque
    ;

bucle_para
    : PARA PAR_IZQ declaracion expr PUNTOYCOMA actualizacion PAR_DER bloque
    ;

actualizacion
    : IDENTIFICADOR ASIGNACION expr
    ;

bloque
    : LLAVE_IZQ instruccion* LLAVE_DER
    ;

expr
    : expr op=('+'|'-'|'*'|'/') expr
    | expr op=('&&'|'||') expr
    | expr op=(MENOR|MAYOR|IGUAL|DISTINTO|MENOR_IGUAL|MAYOR_IGUAL) expr
    | PAR_IZQ expr PAR_DER
    | funcion_llamada
    | IDENTIFICADOR
    | NUMERO
    | CADENA
    ;

funcion_llamada
    : (RAIZ | SIN | COS | POTENCIA) PAR_IZQ expr (',' expr)? PAR_DER
    ;

retorno
    : RETORNAR expr PUNTOYCOMA
    ;

tipo_dato
    : ENTERO
    | DECIMAL
    | TEXTO
    ;

// ======== Lexer ========

// Palabras clave
IMPRIMIR: 'imprimir' ;
SI: 'si' ;
SINO: 'sino' ;
MIENTRAS: 'mientras' ;
PARA: 'para' ;
RETORNAR: 'retornar' ;

RAIZ: 'raiz' ;
SIN: 'sin' ;
COS: 'cos' ;
POTENCIA: 'potencia' ;

// Tipos de datos
ENTERO: 'entero' ;
DECIMAL: 'decimal' ;
TEXTO: 'texto' ;

// Identificadores y literales
IDENTIFICADOR: [a-zA-Z_] [a-zA-Z0-9_]* ;
NUMERO: [0-9]+ ('.' [0-9]+)? ;
CADENA: '"' (~["\r\n])* '"' ;

// Símbolos y operadores
ASIGNACION: '=' ;
PUNTOYCOMA: ';' ;
PAR_IZQ: '(' ;
PAR_DER: ')' ;
LLAVE_IZQ: '{' ;
LLAVE_DER: '}' ;

// Operadores relacionales
MENOR: '<' ;
MAYOR: '>' ;
IGUAL: '==' ;
DISTINTO: '!=' ;
MENOR_IGUAL: '<=' ;
MAYOR_IGUAL: '>=' ;

// Ignorar espacios y comentarios
ESPACIOS: [ \t\r\n]+ -> skip ;
COMENTARIO: '//' ~[\r\n]* -> skip ;

ERROR: . ;

Overwriting LenguajeEdu.g4


In [ ]:
# Generar el lexer y parser desde la gramática usando ANTLR
!java -jar antlr-4.9.2-complete.jar -Dlanguage=Python3 -visitor LenguajeEdu.g4

In [ ]:
# Es útil para verificar que ANTLR haya generado correctamente los archivos Python de la gramática:
!ls LenguajeEdu*.py

LenguajeEduLexer.py	LenguajeEduParser.py
LenguajeEduListener.py	LenguajeEduVisitor.py


### ¿Qué representan los archivos generados al compilar `.g4`?

| Archivo               | Descripción |
|-----------------------|-------------|
| LenguajeEduLexer.py   | Analizador léxico, define los tokens. |
| LenguajeEduParser.py  | Analizador sintáctico, genera el AST. |
| LenguajeEduVisitor.py | Interfaz para recorrer el AST (Visitor). |
| LenguajeEduListener.py| Interfaz Listener (eventos al recorrer el AST). |

##Fase 1: Análisis Léxico
El analizador léxico divide el código en tokens: unidades mínimas como palabras clave, identificadores, operadores, números, etc.

- Herramienta utilizada: ANTLR
- Archivo relevante: LenguajeEduLexer.py

¿Qué archivo genera ANTLR aquí?
- LenguajeEduLexer.py: Define las reglas de los tokens reconocidos.

In [ ]:
import antlr4
from LenguajeEduLexer import LenguajeEduLexer
from antlr4.error.ErrorListener import ErrorListener

class CustomErrorListener(ErrorListener):
    def syntaxError(self, recognizer, offendingSymbol, line, column, msg, e):
        print(f"Error léxico en línea {line}:{column} - {msg}")
        exit(1)

def analyze_tokens(input_file):
    input_stream = antlr4.FileStream(input_file)
    lexer = LenguajeEduLexer(input_stream)
    lexer.removeErrorListeners()
    lexer.addErrorListener(CustomErrorListener())
    token = lexer.nextToken()
    while token.type != antlr4.Token.EOF:
        token_name = LenguajeEduLexer.symbolicNames[token.type]
        print(f"Línea {token.line}, Columna {token.column}: {token_name} ('{token.text}')")
        token = lexer.nextToken()

if __name__ == "__main__":
    with open("test.edu", "w") as f:
        f.write("""
        entero contador = 0;
        decimal pi = 3.14159;
        texto mensaje = "Resultado: ";

        imprimir(mensaje);
        mientras (contador < 3) {
            decimal seno = sin(pi * contador);
            imprimir(seno);
            contador = contador + 1;
        }
        """)

    print("Tokens generados:")
    analyze_tokens("test.edu")

Tokens generados:
Línea 2, Columna 8: PUNTOYCOMA ('entero')
Línea 2, Columna 15: LLAVE_IZQ ('contador')
Línea 2, Columna 24: MAYOR ('=')
Línea 2, Columna 26: LLAVE_DER ('0')
Línea 2, Columna 27: IGUAL (';')
Línea 3, Columna 8: PAR_IZQ ('decimal')
Línea 3, Columna 16: LLAVE_IZQ ('pi')
Línea 3, Columna 19: MAYOR ('=')
Línea 3, Columna 21: LLAVE_DER ('3.14159')
Línea 3, Columna 28: IGUAL (';')
Línea 4, Columna 8: PAR_DER ('texto')
Línea 4, Columna 14: LLAVE_IZQ ('mensaje')
Línea 4, Columna 22: MAYOR ('=')
Línea 4, Columna 24: MENOR ('"Resultado: "')
Línea 4, Columna 37: IGUAL (';')
Línea 6, Columna 8: SIN ('imprimir')
Línea 6, Columna 16: DISTINTO ('(')
Línea 6, Columna 17: LLAVE_IZQ ('mensaje')
Línea 6, Columna 24: MENOR_IGUAL (')')
Línea 6, Columna 25: IGUAL (';')
Línea 7, Columna 8: ENTERO ('mientras')
Línea 7, Columna 17: DISTINTO ('(')
Línea 7, Columna 18: LLAVE_IZQ ('contador')
Línea 7, Columna 27: COMENTARIO ('<')
Línea 7, Columna 29: LLAVE_DER ('3')
Línea 7, Columna 30: MENOR_IGUA

##2. Análisis Sintáctico

Aquí se construye el Árbol de Sintaxis Abstracta (AST), comprobando si la secuencia de tokens es válida según la gramática definida.

Archivo relevante:
- LenguajeEduParser.py

Archivos Generados:
- LenguajeEduParser.py: Define las reglas de la gramática del lenguaje.
- LenguajeEdu.tokens: Contiene la lista de tokens reconocidos.

In [ ]:
import antlr4
from antlr4 import *
from LenguajeEduLexer import LenguajeEduLexer
from LenguajeEduParser import LenguajeEduParser
from antlr4.error.ErrorListener import ErrorListener

class CustomErrorListener(ErrorListener):
    def syntaxError(self, recognizer, offendingSymbol, line, column, msg, e):
        print(f"Error sintáctico en línea {line}:{column} - {msg}")
        exit(1)

def parse_program(input_file):
    input_stream = FileStream(input_file)
    lexer = LenguajeEduLexer(input_stream)
    lexer.removeErrorListeners()
    lexer.addErrorListener(CustomErrorListener())
    token_stream = CommonTokenStream(lexer)
    parser = LenguajeEduParser(token_stream)
    parser.removeErrorListeners()
    parser.addErrorListener(CustomErrorListener())
    tree = parser.programa()
    print("Árbol sintáctico generado:")
    print(tree.toStringTree(recog=parser))
    return tree

if __name__ == "__main__":
    with open("test.edu", "w") as f:
        f.write("""
        entero contador = 0;
        decimal pi = 3.14159;
        texto mensaje = "Resultado: ";

        imprimir(mensaje);
        mientras (contador < 3) {
            decimal seno = sin(pi * contador);
            imprimir(seno);
            contador = contador + 1;
        }
        """)

    print("Iniciando análisis sintáctico...")
    parse_program("test.edu")

Iniciando análisis sintáctico...
Árbol sintáctico generado:
(programa (instruccion (declaracion (tipo_dato entero) contador = (expr 0) ;)) (instruccion (declaracion (tipo_dato decimal) pi = (expr 3.14159) ;)) (instruccion (declaracion (tipo_dato texto) mensaje = (expr "Resultado: ") ;)) (instruccion (impresion imprimir ( (expr mensaje) ) ;)) (instruccion (bucle_mientras mientras ( (expr (expr contador) < (expr 3)) ) (bloque { (instruccion (declaracion (tipo_dato decimal) seno = (expr (funcion_llamada sin ( (expr (expr pi) * (expr contador)) ))) ;)) (instruccion (impresion imprimir ( (expr seno) ) ;)) (instruccion (asignacion contador = (expr (expr contador) + (expr 1)) ;)) }))) <EOF>)


In [ ]:
import antlr4
from antlr4 import *
from LenguajeEduLexer import LenguajeEduLexer
from LenguajeEduParser import LenguajeEduParser
from antlr4.error.ErrorListener import ErrorListener
from graphviz import Digraph
from antlr4.tree.Tree import TerminalNodeImpl

# Custom error listener for syntax errors
class CustomErrorListener(ErrorListener):
    def syntaxError(self, recognizer, offendingSymbol, line, column, msg, e):
        print(f"Error sintáctico en línea {line}:{column} - {msg}")
        exit(1)

# Función para convertir el árbol sintáctico en un grafo Graphviz
def tree_to_graphviz(tree, parser, graph=None, parent=None, node_id=0):
    if graph is None:
        graph = Digraph(comment="Árbol Sintáctico")

    # Determinar si el nodo es terminal o no
    if isinstance(tree, TerminalNodeImpl):
        node_label = tree.getText()  # Usar el texto del token para nodos terminales
    else:
        # Para nodos de reglas, usar el nombre de la regla si está disponible
        node_label = parser.ruleNames[tree.getRuleIndex()] if tree.getRuleIndex() >= 0 else tree.getText()

    node_id_str = str(node_id)

    # Agregar nodo al grafo
    graph.node(node_id_str, label=node_label.replace('"', '\\"'))

    # Conectar con el nodo padre
    if parent is not None:
        graph.edge(parent, node_id_str)

    # Recorrer hijos
    child_count = tree.getChildCount()
    for i in range(child_count):
        child = tree.getChild(i)
        node_id += 1
        node_id = tree_to_graphviz(child, parser, graph, node_id_str, node_id)

    return node_id + 1

def parse_and_visualize(input_file, output_file="syntactic_tree"):
    # Leer el archivo de entrada
    input_stream = FileStream(input_file)

    # Configurar el lexer
    lexer = LenguajeEduLexer(input_stream)
    lexer.removeErrorListeners()
    lexer.addErrorListener(CustomErrorListener())

    # Generar el flujo de tokens
    token_stream = CommonTokenStream(lexer)

    # Configurar el parser
    parser = LenguajeEduParser(token_stream)
    parser.removeErrorListeners()
    parser.addErrorListener(CustomErrorListener())

    # Generar el árbol sintáctico
    tree = parser.programa()

    # Crear el grafo
    graph = Digraph(comment="Árbol Sintáctico")
    tree_to_graphviz(tree, parser, graph)

    # Guardar el grafo como PNG
    graph.render(output_file, format="png", cleanup=True)
    print(f"Árbol sintáctico guardado en {output_file}.png")

if __name__ == "__main__":
    # Instalar graphviz si es necesario
    try:
        from graphviz import Digraph
    except ImportError:
        import os
        os.system("pip install graphviz")
        from graphviz import Digraph

    # Crear archivo de prueba
    with open("test.edu", "w") as f:
        f.write("""
        entero contador = 0;
        decimal pi = 3.14159;
        texto mensaje = "Resultado: ";

        imprimir(mensaje);
        mientras (contador < 3) {
            decimal seno = sin(pi * contador);
            imprimir(seno);
            contador = contador + 1;
        }
        """)

    # Ejecutar el análisis sintáctico y generar la visualización
    print("Iniciando análisis sintáctico y visualización...")
    parse_and_visualize("test.edu")

Iniciando análisis sintáctico y visualización...
Árbol sintáctico guardado en syntactic_tree.png


## Fase 3: Análisis Semántico

En esta etapa verificamos el **significado correcto** del código. Por ejemplo:
- ¿Las variables están declaradas antes de ser usadas?
- ¿Se usan tipos de datos correctos en las operaciones?

### ¿Por qué usamos Visitor aquí?
El patrón Visitor permite recorrer el AST de forma estructurada y controlar cómo se analiza cada tipo de nodo. Es ideal para:
- Construir la **tabla de símbolos**.
- Verificar reglas semánticas personalizadas.

### Archivos Generados:
- `LenguajeEduVisitor.py`: Define la interfaz Visitor.


----------

¿Cuál es la diferencia entre Visitor y Listener?

| Visitor             | Listener           |
|---------------------|--------------------|
| Control total del recorrido. | Eventos automáticos en enter/exit de cada nodo. |
| Recomendado para análisis semántico y generación de código. | Útil para análisis superficial o extracción de información. |


¿Por qué se programan los métodos de semántica y generación de código en el Visitor?

- Permite **controlar el recorrido** del AST de forma explícita.
- Facilita aplicar **acciones específicas** en cada tipo de nodo.
- Es más adecuado para tareas complejas como análisis semántico y transformación del AST en código de salida.

In [ ]:
from LenguajeEduVisitor import LenguajeEduVisitor

print(dir(LenguajeEduVisitor))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'aggregateResult', 'defaultResult', 'shouldVisitNextChild', 'visit', 'visitActualizacion', 'visitAsignacion', 'visitBloque', 'visitBucle_mientras', 'visitBucle_para', 'visitChildren', 'visitCondicional', 'visitDeclaracion', 'visitErrorNode', 'visitExpr', 'visitFuncion_llamada', 'visitImpresion', 'visitInstruccion', 'visitPrograma', 'visitRetorno', 'visitTerminal', 'visitTipo_dato']


In [ ]:
import sys
import antlr4
from antlr4 import *
from LenguajeEduLexer import LenguajeEduLexer
from LenguajeEduParser import LenguajeEduParser
from antlr4.error.ErrorListener import ErrorListener
import os
from collections import defaultdict
import math

# Listener personalizado para errores durante el análisis léxico y sintáctico
class CustomErrorListener(ErrorListener):
    def syntaxError(self, recognizer, offendingSymbol, line, column, msg, e):
        print(f"Error sintáctico en línea {line}:{column} - {msg}")
        sys.exit(1)

# Visitor para análisis semántico y generación de código
class EduVisitor(ParseTreeVisitor):
    def __init__(self):
        super().__init__()
        self.symbol_table = SymbolTable()
        self.code_gen = CodeGenerator()
        self.current_type = None

    def visit(self, tree):
        print(f"[DEBUG] Visiting node of type: {type(tree).__name__}")
        if isinstance(tree, LenguajeEduParser.ProgramaContext):
            return self.visitPrograma(tree)
        elif isinstance(tree, LenguajeEduParser.InstruccionContext):
            return self.visitInstruccion(tree)
        elif isinstance(tree, LenguajeEduParser.DeclaracionContext):
            return self.visitDeclaracion(tree)
        elif isinstance(tree, LenguajeEduParser.AsignacionContext):
            return self.visitAsignacion(tree)
        elif isinstance(tree, LenguajeEduParser.ImpresionContext):
            return self.visitImpresion(tree)
        elif isinstance(tree, LenguajeEduParser.CondicionalContext):
            return self.visitCondicional(tree)
        elif isinstance(tree, LenguajeEduParser.Bucle_mientrasContext):
            return self.visitBucle_mientras(tree)
        elif isinstance(tree, LenguajeEduParser.Bucle_paraContext):
            return self.visitBucle_para(tree)
        elif isinstance(tree, LenguajeEduParser.ActualizacionContext):
            return self.visitActualizacion(tree)
        elif isinstance(tree, LenguajeEduParser.ExprContext):
            return self.visitExpr(tree)
        elif isinstance(tree, LenguajeEduParser.Funcion_llamadaContext):
            return self.visitFuncion_llamada(tree)
        elif isinstance(tree, LenguajeEduParser.RetornoContext):
            return self.visitRetorno(tree)
        elif isinstance(tree, LenguajeEduParser.BloqueContext):
            return self.visitBloque(tree)
        return super().visit(tree)

    def visitPrograma(self, ctx):
        print("[DEBUG] Visiting programa")
        try:
            for instr in ctx.instruccion():
                print(f"[DEBUG] Visiting instruccion: {type(instr).__name__}")
                self.visit(instr)
            self.symbol_table.check_unused()
            self.symbol_table.print_table()
            code = self.code_gen.get_code()
            print(f"[DEBUG] Generated code: {code}")
            return code
        except Exception as e:
            print(f"[ERROR] Exception in visitPrograma: {str(e)}")
            raise

    def visitInstruccion(self, ctx):
        print("[DEBUG] Visiting instruccion")
        return self.visit(ctx.getChild(0))

    def visitBloque(self, ctx):
        print("[DEBUG] Visiting bloque")
        for child in ctx.instruccion():
            self.visit(child)
        return None

    def visitDeclaracion(self, ctx):
        print("[DEBUG] Visiting declaracion")
        var_name = ctx.IDENTIFICADOR().getText()
        type_ = ctx.tipo_dato().getText()
        line = ctx.start.line
        self.symbol_table.declare(var_name, type_, line)
        if ctx.expr():
            value = self.visit(ctx.expr())
            self.code_gen.emit(f"{var_name} = {value}")
        return None

    def visitAsignacion(self, ctx):
        print("[DEBUG] Visiting asignacion")
        var_name = ctx.IDENTIFICADOR().getText()
        self.symbol_table.use(var_name, ctx.start.line)
        value = self.visit(ctx.expr())
        self.code_gen.emit(f"{var_name} = {value}")
        return None

    def visitImpresion(self, ctx):
        print("[DEBUG] Visiting impresion")
        if ctx.expr():
            value = self.visit(ctx.expr())
            self.code_gen.emit(f"print({value})")
        else:
            value = ctx.CADENA().getText()
            self.code_gen.emit(f"print({value})")
        return None

    def visitCondicional(self, ctx):
        print("[DEBUG] Visiting condicional")
        condition = self.visit(ctx.expr())
        label_else = self.code_gen.new_temp()
        label_end = self.code_gen.new_temp()
        self.code_gen.emit(f"if not {condition} goto {label_else}")
        self.visit(ctx.bloque(0))
        if ctx.SINO():
            self.code_gen.emit(f"goto {label_end}")
        self.code_gen.emit(f"{label_else}:")
        if ctx.SINO():
            self.visit(ctx.bloque(1))
        self.code_gen.emit(f"{label_end}:")
        return None

    def visitBucle_mientras(self, ctx):
        print("[DEBUG] Visiting bucle_mientras")
        label_start = self.code_gen.new_temp()
        label_end = self.code_gen.new_temp()
        self.code_gen.emit(f"{label_start}:")
        condition = self.visit(ctx.expr())
        self.code_gen.emit(f"if not {condition} goto {label_end}")
        self.visit(ctx.bloque())
        self.code_gen.emit(f"goto {label_start}")
        self.code_gen.emit(f"{label_end}:")
        return None

    def visitBucle_para(self, ctx):
        print("[DEBUG] Visiting bucle_para")
        self.visit(ctx.declaracion())
        label_start = self.code_gen.new_temp()
        label_end = self.code_gen.new_temp()
        self.code_gen.emit(f"{label_start}:")
        condition = self.visit(ctx.expr())
        self.code_gen.emit(f"if not {condition} goto {label_end}")
        self.visit(ctx.bloque())
        self.visit(ctx.actualizacion())
        self.code_gen.emit(f"goto {label_start}")
        self.code_gen.emit(f"{label_end}:")
        return None

    def visitActualizacion(self, ctx):
        print("[DEBUG] Visiting actualizacion")
        var_name = ctx.IDENTIFICADOR().getText()
        self.symbol_table.use(var_name, ctx.start.line)
        value = self.visit(ctx.expr())
        self.code_gen.emit(f"{var_name} = {value}")
        return None

    def visitExpr(self, ctx):
        print("[DEBUG] Visiting expr")
        if ctx.op:
            left = self.visit(ctx.expr(0))
            right = self.visit(ctx.expr(1))
            op = ctx.op.text
            if op == '/' and right == '0':
                print(f"Error semántico en línea {ctx.start.line}: División por cero detectada")
                sys.exit(1)
            temp = self.code_gen.new_temp()
            self.code_gen.emit(f"{temp} = {left} {op} {right}")
            return temp
        elif ctx.IDENTIFICADOR():
            var_name = ctx.IDENTIFICADOR().getText()
            self.symbol_table.use(var_name, ctx.start.line)
            return var_name
        elif ctx.NUMERO():
            return ctx.NUMERO().getText()
        elif ctx.CADENA():
            return ctx.CADENA().getText()
        elif ctx.funcion_llamada():
            return self.visit(ctx.funcion_llamada())
        else:
            return self.visit(ctx.expr(0))

    def visitFuncion_llamada(self, ctx):
        print("[DEBUG] Visiting funcion_llamada")
        func_name = ctx.getChild(0).getText()
        args = [self.visit(ctx.expr(i)) for i in range(len(ctx.expr()))]
        temp = self.code_gen.new_temp()
        if func_name == 'raiz':
            self.code_gen.emit(f"{temp} = math.sqrt({args[0]})")
        elif func_name == 'sin':
            self.code_gen.emit(f"{temp} = math.sin({args[0]})")
        elif func_name == 'cos':
            self.code_gen.emit(f"{temp} = math.cos({args[0]})")
        elif func_name == 'potencia':
            self.code_gen.emit(f"{temp} = math.pow({args[0]}, {args[1]})")
        return temp

    def visitRetorno(self, ctx):
        print("[DEBUG] Visiting retorno")
        value = self.visit(ctx.expr())
        self.code_gen.emit(f"return {value}")
        return None

def compile_program(input_file):
    print(f"[DEBUG] Compiling {input_file}")
    try:
        input_stream = FileStream(input_file)
        lexer = LenguajeEduLexer(input_stream)
        lexer.removeErrorListeners()
        lexer.addErrorListener(CustomErrorListener())
        token_stream = CommonTokenStream(lexer)
        parser = LenguajeEduParser(token_stream)
        parser.removeErrorListeners()
        parser.addErrorListener(CustomErrorListener())
        tree = parser.programa()
        print(f"[DEBUG] Tree type: {type(tree).__name__}")
        visitor = EduVisitor()
        code = visitor.visit(tree)
        print(f"[DEBUG] Code returned: {code}")
        return code
    except Exception as e:
        print(f"[ERROR] Exception in compile_program: {str(e)}")
        raise


## Tabla de Símbolos

La **tabla de símbolos** es una estructura de datos utilizada durante el análisis semántico de un compilador. Su función principal es **almacenar y gestionar la información sobre los identificadores** (variables, métodos, clases, etc.) que aparecen en el código fuente.

## ¿Por qué es importante?

Permite:
- Verificar que las variables estén **declaradas antes de ser utilizadas**.
- Controlar que no haya **declaraciones duplicadas** de variables o métodos.
- Validar **tipos de datos** durante las operaciones.
- Gestionar correctamente los **alcances (scopes)** de las variables y métodos.

---

## Ejemplo Conceptual

Considera este código:

```java
class Test {
    int x;
    public int main() {
        int y;
        y = 5;
        return y;
    }
}

La tabla de símbolos se construiría así:

| Clase | Tipo | Nombre | Ámbito |
| ----- | ---- | ------ | ------ |
| Test  | int  | x      | Global |
| Test  | int  | y      | main() |

¿Cómo lo implementamos en Python?
Usamos un diccionario de diccionarios:

symbol_table = {
    "Test": {  # Clase
        "x": "int",  # Variable global de la clase
        "main": {
            "y": "int"  # Variable local al método main
        }
    }
}


In [ ]:
# Tabla de símbolos para análisis semántico
class SymbolTable:
    def __init__(self):
        self.symbols = {}
        self.used_vars = set()

    def declare(self, name, type_, line):
        if name in self.symbols:
            print(f"Error semántico en línea {line}: Variable '{name}' ya declarada")
            sys.exit(1)
        if not name[0].islower():
            print(f"Error semántico en línea {line}: Nombre de variable '{name}' debe comenzar con minúscula")
            sys.exit(1)
        self.symbols[name] = {'type': type_, 'line': line}

    def use(self, name, line):
        if name not in self.symbols:
            print(f"Error semántico en línea {line}: Variable '{name}' no declarada")
            sys.exit(1)
        self.used_vars.add(name)

    def check_unused(self):
        for name, info in self.symbols.items():
            if name not in self.used_vars:
                print(f"Advertencia en línea {info['line']}: Variable '{name}' declarada pero no usada")

    def print_table(self):
        print("\nTabla de Símbolos:")
        for name, info in self.symbols.items():
            print(f"  → {name}: tipo={info['type']}, línea={info['line']}")



# Fase 4: Generación de Código

Finalmente, a partir del AST y las validaciones semánticas, generamos un **código de salida**.

En este caso, generamos:
- **Python**.
- **Pseudocódigo**: Una representación abstracta entendible para humanos.

¿Por qué usamos Visitor también aquí?
- Permite recorrer el AST y decidir cómo convertir cada nodo en instrucciones del código de salida.


In [ ]:
# Generación de código
class CodeGenerator:
    def __init__(self):
        self.code = []
        self.temp_count = 0

    def new_temp(self):
        self.temp_count += 1
        return f"t{self.temp_count}"

    def emit(self, instruction):
        self.code.append(instruction)
        print(f"[DEBUG] Emitting: {instruction}")

    def get_code(self):
        return "\n".join(self.code)



if __name__ == "__main__":
    try:
        with open("test.edu", "w") as f:
            f.write("""
            entero contador = 0;
            decimal pi = 3.14159;
            texto mensaje = "Resultado: ";

            imprimir(mensaje);
            mientras (contador < 3) {
                decimal seno = sin(pi * contador);
                imprimir(seno);
                contador = contador + 1;
            }
            """)

        print("Iniciando compilación...")
        generated_code = compile_program("test.edu")
        print("Código intermedio generado:")
        print(generated_code if generated_code else "[ERROR] No code generated")
    except Exception as e:
        print(f"[ERROR] Exception in main: {str(e)}")


Iniciando compilación...
[DEBUG] Compiling test.edu
[DEBUG] Tree type: ProgramaContext
[DEBUG] Visiting node of type: ProgramaContext
[DEBUG] Visiting programa
[DEBUG] Visiting instruccion: InstruccionContext
[DEBUG] Visiting node of type: InstruccionContext
[DEBUG] Visiting instruccion
[DEBUG] Visiting node of type: DeclaracionContext
[DEBUG] Visiting declaracion
[DEBUG] Visiting node of type: ExprContext
[DEBUG] Visiting expr
[DEBUG] Emitting: contador = 0
[DEBUG] Visiting instruccion: InstruccionContext
[DEBUG] Visiting node of type: InstruccionContext
[DEBUG] Visiting instruccion
[DEBUG] Visiting node of type: DeclaracionContext
[DEBUG] Visiting declaracion
[DEBUG] Visiting node of type: ExprContext
[DEBUG] Visiting expr
[DEBUG] Emitting: pi = 3.14159
[DEBUG] Visiting instruccion: InstruccionContext
[DEBUG] Visiting node of type: InstruccionContext
[DEBUG] Visiting instruccion
[DEBUG] Visiting node of type: DeclaracionContext
[DEBUG] Visiting declaracion
[DEBUG] Visiting node of ty